In [1]:
import pickle as pkl
import nltk
import os
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
import multiprocessing
import evaluate
from rouge_score import rouge_scorer
from tqdm.auto import tqdm
from natsort import natsorted
import random
import pandas as pd
from datasets import load_dataset, DatasetDict, load_from_disk

/home/haskari/miniconda3/envs/acl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-24 18:30:26,392] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
pkls=os.listdir('cnn')
pkls=natsorted(pkls)
pkls_list=[]
for pikl in pkls:
    with open('cnn/{}'.format(pikl),'rb') as f:
        file=pkl.load(f)
    if not file:
        file.append(' ')
    pkls_list.extend(file)

print(len(pkls_list))
print(pkls_list[0])

11490
 Sure! Here is a summary of the article in three numbered sentences:

1. The Palestinian Authority officially became the 123rd member of the International Criminal Court (ICC) on Wednesday, giving the court jurisdiction over alleged crimes in Palestinian territories.
2. The ICC has initiated a preliminary examination into the situation in the Palestinian territories, potentially paving the way for investigations into possible war crimes committed by Israelis and Palestinians.
3. The accession to the ICC is seen as a move towards greater justice and accountability for alleged crimes committed in the occupied Palestinian territories, but both Israel and the United States have objected to the Palestinians' attempts to join the court.


In [3]:
count=0
for idx,item in enumerate(pkls_list):
    if item == " ":
        print(idx)
        count+=1
print(count)

13
29
56
98
187
197
216
236
237
242
259
297
304
306
347
364
397
404
408
409
430
437
456
492
493
574
582
678
716
725
727
770
805
817
827
841
842
861
865
902
904
993
1011
1018
1036
1037
1097
1221
1292
1349
1401
1505
1546
1566
1587
1673
1678
1680
2021
2024
2123
2227
2303
2325
2326
2373
2661
2663
2703
2764
2796
2958
2965
2975
2981
3111
3141
3247
3314
3329
3349
3417
3500
3574
3590
3779
4071
4168
4216
4342
4354
4464
4475
4483
4596
4632
4643
4658
4659
4662
4857
4865
4870
4884
4886
4941
4943
4949
4973
4978
4980
4987
4988
5010
5018
5043
5065
5070
5075
5109
5110
5147
5157
5178
5182
5185
5197
5198
5205
5211
5215
5228
5237
5240
5246
5247
5259
5270
5330
5343
5346
5373
5376
5395
5404
5407
5414
5427
5456
5469
5470
5471
5500
5507
5526
5579
5580
5602
5604
5605
5634
5641
5642
5668
5719
5730
5740
5765
5768
5770
5780
5793
5807
5814
5881
5893
5913
5919
5969
6000
6014
6054
6074
6083
6084
6104
6107
6115
6137
6181
6193
6212
6215
6225
6248
6255
6256
6264
6346
6347
6354
6361
6362
6378
6389
6395
6429
6458
6498
6

In [4]:
cnn_list=[]
for i in range(len(pkls_list)):
    temp=[]
    splits=pkls_list[i].split('\n')
    for split in splits:
        if split:
            if split[0].isdigit():
                try:
                    temp.append(split.split(".")[1].strip())
                except:
                    print(split)
    cnn_list.append(temp)

In [5]:
len(cnn_list)

11490

In [6]:
cnn_list[200]

['Former New England Patriots player Aaron Hernandez is facing three more legal proceedings, including one criminal trial and two civil lawsuits, after being convicted of murder and other charges in the death of Odin Lloyd',
 'The double-murder trial for the deaths of Daniel de Abreu and Safiro Furtado is set to resume, with a new court date to be determined, and Hernandez is also facing civil suits from the families of the victims and a man who claims Hernandez shot him in a Miami strip club',
 "Hernandez's legal team has stated that he was unable to effectively defend himself during his trial in Massachusetts due to various factors, and the NFL players union has filed a grievance over unpaid bonus money on behalf of Hernandez"]

In [34]:
# with open('reddit.pkl', 'wb') as f:
#     pkl.dump(cnn_list,f) 

In [3]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'
dataset=dataset['test']

In [2]:
dataset = load_dataset("EdinburghNLP/xsum")
article_key = 'document'
summary_key = 'summary'
dataset=dataset['test']

In [290]:
dataset = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
dataset = DatasetDict({
    'train': dataset['test'],
    'test': dataset['train']})

dataset=dataset['test']

In [341]:
data = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
    # 80% train, 20% test + validation
train_testvalid = data['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

dataset=dataset['test']

In [144]:
articles=dataset.select(range(10))[article_key]

In [30]:
articles=dataset.select([2,5,7,8,9,13,14,15,18,21])[article_key]

In [323]:
articles=dataset.select([1,2,4,5,6,7,11,12,14,15])[article_key]

In [4]:
articles

['Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for ac

In [31]:
for k,v in enumerate(articles):
    articles[k]=v.replace('\n',"")

In [326]:
gold=dataset.select(range(10))[summary_key]

In [32]:
gold=dataset.select([2,5,7,8,9,13,14,15,18,21])[summary_key]

In [327]:
gold=dataset.select([1,2,4,5,6,7,11,12,14,15])[summary_key]

In [346]:
gold_new=[]
for i in range(10):
    print(gold[i][0]['text'])
    gold_new.append(gold[i][0]['text'])
    

TypeError: string indices must be integers, not 'str'

In [33]:
for k,v in enumerate(gold):
    gold[k]=v.replace('\n',"")

In [329]:
for k,v in enumerate(gold_new):
    gold_new[k]=v.replace('\n',"")

In [330]:
gold_new

["U.N. chief calls for united, 'appropriate action' on North Korea",
 'House tax positions begin to emerge ahead of talks with Senate',
 'Trump strongly committed to partnership with EU, says VP Pence',
 "Commentary: Trump can't fight Islamic State without wading into Syria's war",
 'Top Senate Republican tells Supreme Court nominee: Senate will not act',
 'Ex-Michigan lawmaker to stand trial; charges dismissed against another',
 'Iceland sets snap election for Oct. 28: president',
 'Ex-Georgian leader risks extradition on return to Ukraine',
 "China's Xi lays out vision for 'new era' led by 'still stronger' Communist Party",
 'Armed group seeks legitimacy with Tripoli migrant deal, source says']

In [34]:
gold

['Four people accused of kidnapping and torturing a mentally disabled man in a "racially motivated" attack streamed on Facebook have been denied bail.',
 'Since their impending merger was announced in January, there has been remarkably little comment about the huge proposed deal to combine Essilor and Luxottica.',
 'Have you heard the one about the computer programmer who bought a failing comedy club in Texas and turned it into a million dollar a year business?',
 "The reaction from BT's investors told us much about media regulator Ofcom's ruling on the fate of Openreach, the BT subsidiary that provides much of the UK's broadband infrastructure.",
 "Manager Brendan Rodgers is sure Celtic can exploit the wide open spaces of Hampden when they meet Rangers in Sunday's League Cup semi-final.",
 "Torquay United boss Kevin Nicholson says none of the money from Eunan O'Kane's move to Leeds from Bournemouth will go to the playing squad.",
 'North Wales Police has been criticised at an inquest 

In [35]:
with open('xsum_capped_random.pkl', 'rb') as f:
    para_file=pkl.load(f) 

In [36]:
paraphrased=[]
#for i in range(10):
for i in [2,5,7,8,9,13,14,15,18,21]:
#for i in [1,2,4,5,6,7,11,12,14,15]:
    paraphrased.append(" ".join(para_file[i]))
    

In [37]:
with open('../data_original/xsum_capped_random.pkl', 'rb') as f:
    orig_file=pkl.load(f) 

In [38]:
original=[]
#for i in range(10):
for i in [2,5,7,8,9,13,14,15,18,21]:
#for i in [1,2,4,5,6,7,11,12,14,15]:
    original.append(" ".join(orig_file[i]))

In [39]:
orig_file[9]

['He believes the pitch will suit the speed and intensity of his team']

In [40]:
para_file[9]

["Celtic's previous trip to Hampden Park ended in a loss against Rangers in the Scottish Cup semi-final"]

In [41]:
dic={'Article':articles, 'Paraphrased':paraphrased, 'Original': original, 'Gold':gold }

In [376]:
orig_file[21]

['Luckily, my garbage was still in my bin so i fished it out']

In [42]:
df=pd.DataFrame.from_dict(dic)

In [43]:
df.to_csv('../annotation/mistral-xsum.csv', index=False)

In [69]:
paraphrased

['Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders The Welsh Government said more people than ever were getting help to address housing problems Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority Building more one-bedroom flats could help ease the problem Official figures show 830 one-bedroom properties were built in the year to March 2016, of an overall total of 6,900 new properties in Wales Marc, 50, who has been in and out of prison for the p